# First steps with Linkerd 2.0
In this notebook, we explore the service mesh Linkerd 2.0.

References:
* https://kubernetes.io/blog/2018/09/18/hands-on-with-linkerd-2.0/
* https://linkerd.io/2/getting-started/

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-up-a-local-single-node-Kubernetes-cluster" data-toc-modified-id="Set-up-a-local-single-node-Kubernetes-cluster-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set up a local single-node Kubernetes cluster</a></span></li><li><span><a href="#Set-up-a-demo-application-in-the-Kubernetes-cluster" data-toc-modified-id="Set-up-a-demo-application-in-the-Kubernetes-cluster-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Set up a demo application in the Kubernetes cluster</a></span><ul class="toc-item"><li><span><a href="#Deploy-the-demo-app-emojivoto" data-toc-modified-id="Deploy-the-demo-app-emojivoto-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Deploy the demo app <em>emojivoto</em></a></span></li><li><span><a href="#Find-out-on-which-port-the-service-is-running" data-toc-modified-id="Find-out-on-which-port-the-service-is-running-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Find out on which port the service is running</a></span></li><li><span><a href="#Map-the-port-to-a-port-on-the-host-system-with-`kubectl-port-forward'" data-toc-modified-id="Map-the-port-to-a-port-on-the-host-system-with-`kubectl-port-forward'-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Map the port to a port on the host system with `kubectl port-forward'</a></span></li></ul></li><li><span><a href="#Set-up-Linkderd-2" data-toc-modified-id="Set-up-Linkderd-2-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set up Linkderd 2</a></span><ul class="toc-item"><li><span><a href="#Install-Linkerd-2-in-the-user's-home-directory" data-toc-modified-id="Install-Linkerd-2-in-the-user's-home-directory-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Install Linkerd 2 in the user's home directory</a></span></li><li><span><a href="#Deploy-Linkerd’s-control-plane-to-the-cluster" data-toc-modified-id="Deploy-Linkerd’s-control-plane-to-the-cluster-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Deploy Linkerd’s control plane to the cluster</a></span></li><li><span><a href="#Add-Linkerd-to-the-web-service" data-toc-modified-id="Add-Linkerd-to-the-web-service-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Add Linkerd to the web service</a></span></li></ul></li><li><span><a href="#Shut-down-the-Kubernetes-cluster" data-toc-modified-id="Shut-down-the-Kubernetes-cluster-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Shut down the Kubernetes cluster</a></span></li></ul></div>

## Set up a local single-node Kubernetes cluster
To see how this works, look at [setting-up-k8s-cluster-with-kind.ipynb](setting-up-k8s-cluster-with-kind.ipynb)

In [1]:
. kubernetes-utils
create-kind-cluster linkerd-test

Creating cluster "linkerd-test" ...
 ✓ Ensuring node image (kindest/node:v1.15.3) 🖼
 ✓ Preparing nodes 📦 
 ✓ Creating kubeadm config 📜 
 ✓ Starting control-plane 🕹️ 
 ✓ Installing CNI 🔌 
 ✓ Installing StorageClass 💾 
Cluster creation complete. You can now use the cluster with:

export KUBECONFIG="$(kind get kubeconfig-path --name="linkerd-test")"
kubectl cluster-info


## Set up a demo application in the Kubernetes cluster

### Deploy the demo app *emojivoto*
Note that we use `kubectl rollout status` to wait until the deployments are finished. Unfortunately, we cannot pass `emojivoto.yml` to this command directly because it only supports files which contain exactly one deployment, see, e.g., https://github.com/kubernetes/kubernetes/issues/72216

In [2]:
export EMOJIVOTO=download/emojivoto.yml

if [ ! -f $EMOJIVOTO ]; then
    mkdir -p download
    curl -s https://run.linkerd.io/emojivoto.yml -o $EMOJIVOTO
fi

kubectl apply -f $EMOJIVOTO

kubectl rollout status -n emojivoto deployment emoji
kubectl rollout status -n emojivotodeployment voting
kubectl rollout status -n emojivoto deployment web
kubectl rollout status -n emojivoto deployment vote-bot

namespace/emojivoto created
serviceaccount/emoji created
serviceaccount/voting created
serviceaccount/web created
deployment.apps/emoji created
service/emoji-svc created
deployment.apps/voting created
service/voting-svc created
deployment.apps/web created
service/web-svc created
deployment.apps/vote-bot created
Waiting for deployment "emoji" rollout to finish: 0 of 1 updated replicas are available...
deployment "emoji" successfully rolled out
error: the server doesn't have a resource type "voting"
Waiting for deployment "web" rollout to finish: 0 of 1 updated replicas are available...
deployment "web" successfully rolled out
Waiting for deployment "vote-bot" rollout to finish: 0 of 1 updated replicas are available...
deployment "vote-bot" successfully rolled out


### Find out on which port the service is running

In [3]:
kubectl get all -n emojivoto

NAME                            READY   STATUS    RESTARTS   AGE
pod/emoji-7bc9fb68b4-vt97g      1/1     Running   0          49s
pod/vote-bot-5648545766-ckjjx   1/1     Running   0          49s
pod/voting-7c846d6bb9-snsm5     1/1     Running   0          49s
pod/web-ff65d94cd-zv6tm         1/1     Running   0          49s


NAME                 TYPE           CLUSTER-IP    EXTERNAL-IP   PORT(S)        AGE
service/emoji-svc    ClusterIP      None          <none>        8080/TCP       49s
service/voting-svc   ClusterIP      None          <none>        8080/TCP       49s
service/web-svc      LoadBalancer   10.97.83.10   <pending>     80:31361/TCP   49s


NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/emoji      1/1     1            1           49s
deployment.apps/vote-bot   1/1     1            1           49s
deployment.apps/voting     1/1     1            1           49s
deployment.apps/web        1/1     1            1           49s

NAME              

In [4]:
kubectl get -n emojivoto services

NAME         TYPE           CLUSTER-IP    EXTERNAL-IP   PORT(S)        AGE
emoji-svc    ClusterIP      None          <none>        8080/TCP       50s
voting-svc   ClusterIP      None          <none>        8080/TCP       50s
web-svc      LoadBalancer   10.97.83.10   <pending>     80:31361/TCP   50s


In [5]:
# Note that kubectl port-forward needs the service port *inside the container*
EMOJIVOTO_WEB_PORT=$(kubectl get -n emojivoto services | grep web-svc | sed -E 's/^.*\s([0-9]+):[0-9]+\/TCP.*$/\1/g')
echo $EMOJIVOTO_WEB_PORT

80


### Map the port to a port on the host system with `kubectl port-forward'

In [6]:
# Just choose a port which is hopefully unused on the local system
LOCAL_WEB_PORT=8097
echo "Setting up access to the emoji vote service at http://127.0.0.1:$LOCAL_WEB_PORT"
kubectl port-forward -n emojivoto service/web-svc $LOCAL_WEB_PORT:$EMOJIVOTO_WEB_PORT > /dev/null &

Setting up access to the emoji vote service at http://127.0.0.1:8097
[1] 14103


## Set up Linkderd 2

### Install Linkerd 2 in the user's home directory

In [7]:
if [ ! -d $HOME/.linkerd2 ]; then
    curl -sL https://run.linkerd.io/install | sh
    echo
fi

export PATH=$PATH:$HOME/.linkerd2/bin

linkerd version

Client version: stable-2.5.0
Server version: unavailable


### Deploy Linkerd’s control plane to the cluster

In [8]:
linkerd check --pre

kubernetes-api
--------------
√ can initialize the client
√ can query the Kubernetes API

kubernetes-version
------------------
√ is running the minimum Kubernetes API version
√ is running the minimum kubectl version

pre-kubernetes-setup
--------------------
√ control plane namespace does not already exist
√ can create Namespaces
√ can create ClusterRoles
√ can create ClusterRoleBindings
√ can create CustomResourceDefinitions
√ can create PodSecurityPolicies
√ can create ServiceAccounts
√ can create Services
√ can create Deployments
√ can create CronJobs
√ can create ConfigMaps
√ no clock skew detected

pre-kubernetes-capability
-------------------------
√ has NET_ADMIN capability
√ has NET_RAW capability

pre-linkerd-global-resources
----------------------------
√ no ClusterRoles exist
√ no ClusterRoleBindings exist
√ no CustomResourceDefinitions exist
√ no MutatingWebhookConfigurations exist
√ no ValidatingWebhookConfigurations exist
√ no PodSecurityPolicies exist

linkerd-version
-

In [9]:
linkerd install | kubectl apply -f -

kubectl rollout status -n linkerd deploy linkerd-identity
kubectl rollout status -n linkerd deploy linkerd-controller
kubectl rollout status -n linkerd deploy linkerd-web
kubectl rollout status -n linkerd deploy linkerd-prometheus
kubectl rollout status -n linkerd deploy linkerd-grafana
kubectl rollout status -n linkerd deploy linkerd-proxy-injector
kubectl rollout status -n linkerd deploy linkerd-sp-validator
kubectl rollout status -n linkerd deploy linkerd-tap

namespace/linkerd created
clusterrole.rbac.authorization.k8s.io/linkerd-linkerd-identity created
clusterrolebinding.rbac.authorization.k8s.io/linkerd-linkerd-identity created
serviceaccount/linkerd-identity created
clusterrole.rbac.authorization.k8s.io/linkerd-linkerd-controller created
clusterrolebinding.rbac.authorization.k8s.io/linkerd-linkerd-controller created
serviceaccount/linkerd-controller created
role.rbac.authorization.k8s.io/linkerd-heartbeat created
rolebinding.rbac.authorization.k8s.io/linkerd-heartbeat created
serviceaccount/linkerd-heartbeat created
clusterrolebinding.rbac.authorization.k8s.io/linkerd-linkerd-web-admin created
serviceaccount/linkerd-web created
customresourcedefinition.apiextensions.k8s.io/serviceprofiles.linkerd.io created
customresourcedefinition.apiextensions.k8s.io/trafficsplits.split.smi-spec.io created
clusterrole.rbac.authorization.k8s.io/linkerd-linkerd-prometheus created
clusterrolebinding.rbac.authorization.k8s.io/linkerd-linkerd-prometheus cr

In [10]:
linkerd check

kubernetes-api
--------------
√ can initialize the client
√ can query the Kubernetes API

kubernetes-version
------------------
√ is running the minimum Kubernetes API version
√ is running the minimum kubectl version

linkerd-config
--------------
√ control plane Namespace exists
√ control plane ClusterRoles exist
√ control plane ClusterRoleBindings exist
√ control plane ServiceAccounts exist
√ control plane CustomResourceDefinitions exist
√ control plane MutatingWebhookConfigurations exist
√ control plane ValidatingWebhookConfigurations exist
√ control plane PodSecurityPolicies exist

linkerd-existence
-----------------
√ 'linkerd-config' config map exists
√ control plane replica sets are ready
√ no unschedulable pods
√ controller pod is running
√ can initialize the client
√ can query the control plane API

linkerd-api
-----------
√ control plane pods are ready
√ control plane self-check
√ [kubernetes] control plane can talk to Kubernetes
√ [prometheus] control plane can talk to Prome

In [11]:
linkerd dashboard &
sleep 1 # make sure that the initial output of 'linkerd dashboard' is output below the cell

[2] 21690
Linkerd dashboard available at:
http://127.0.0.1:50750
Grafana dashboard available at:
http://127.0.0.1:50750/grafana
Opening Linkerd dashboard in the default browser


### Add Linkerd to the web service

In [12]:
kubectl get -n emojivoto deploy/web -o yaml | linkerd inject - | kubectl apply -f -


deployment "web" injected

deployment.extensions/web configured


Now, have a look at the Linkerd dashboard and see what information about the service is gathered.

## Shut down the Kubernetes cluster

In [13]:
delete-kind-cluster

Deleting cluster "linkerd-test" ...
E0916 16:37:21.644433   14103 portforward.go:233] lost connection to pod
[1]-  Fertig                  kubectl port-forward -n emojivoto service/web-svc $LOCAL_WEB_PORT:$EMOJIVOTO_WEB_PORT > /dev/null
[2]+  Fertig                  linkerd dashboard
